# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# extract and import
orders = pd.read_csv('Orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
#Sub Problem 1: How to aggregate the  amount_spent for unique customers?
amount_by_customer = orders.groupby(['CustomerID','Country'])['amount_spent'].sum()
amount_by_customer= amount_by_customer.reset_index()
amount_by_customer

,CustomerID,Country,amount_spent
0,12346,United Kingdom,77183.60
1,12347,Iceland,4310.00
2,12348,Finland,1797.24
3,12349,Italy,1757.55
4,12350,Norway,334.40
...,...,...,...
4342,18280,United Kingdom,180.60
4343,18281,United Kingdom,80.82
4344,18282,United Kingdom,178.05
4345,18283,United Kingdom,2094.88


In [4]:
#Sub Problem 2/3:
#VIP (95th percentile):
percentiles = np.percentile(amount_by_customer['amount_spent'], [75, 95])
VIP_Customers = amount_by_customer[amount_by_customer['amount_spent'] >= percentiles[1]]['CustomerID']

#Preferred (between the 75th and 95th percentile):
Preferred_Customers = amount_by_customer[(percentiles[1] > amount_by_customer['amount_spent']) & (amount_by_customer['amount_spent'] >= percentiles[0])]['CustomerID']

Customer_Type =[]

for i in range(0,len(amount_by_customer)):
    if (amount_by_customer['amount_spent'].iloc[i] >= percentiles[1]):
        Customer_Type.append('VIP')
    elif (amount_by_customer['amount_spent'].iloc[i] >= percentiles[0]):
        Customer_Type.append('Preferred')
    else:
        Customer_Type.append('Normal')
        
amount_by_customer['Customer_Type'] = Customer_Type
amount_by_customer

,CustomerID,Country,amount_spent,Customer_Type
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,Preferred
2,12348,Finland,1797.24,Preferred
3,12349,Italy,1757.55,Preferred
4,12350,Norway,334.40,Normal
...,...,...,...,...
4342,18280,United Kingdom,180.60,Normal
4343,18281,United Kingdom,80.82,Normal
4344,18282,United Kingdom,178.05,Normal
4345,18283,United Kingdom,2094.88,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [24]:
#Group the countrys and count the number of VIP Customers:
country_vip = amount_by_customer[amount_by_customer['Customer_Type'] == 'VIP'].groupby(['Country'])['Customer_Type'].count().sort_values(ascending=False).reset_index()

#amount_by_customer[amount_by_customer['amount_spent'] >= percentiles[1]]['CustomerID']
country_vip.head(1)

,Country,Customer_Type
0,United Kingdom,178


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [54]:
# Group by with the 'Preferred Customers
country_vip_p = amount_by_customer[amount_by_customer['Customer_Type'] == 'Preferred'].groupby(['Country'])['Customer_Type'].count().sort_values(ascending=False).reset_index()
# Rename the columns
country_vip_p.columns = ['Country', 'Preferred']
country_vip_p = country_vip_p.join(country_vip.set_index('Country'), on='Country')


In [56]:
country_vip_p.columns = ['Country', 'Preferred', 'VIP']

In [58]:
country_vip_p

,Country,Preferred,VIP
0,United Kingdom,756,178.0
1,Germany,28,11.0
2,France,20,9.0
3,Belgium,11,NaN
4,Switzerland,6,3.0
5,Norway,6,1.0
6,Spain,5,2.0
7,Portugal,5,2.0
8,Italy,5,NaN
9,Finland,4,1.0
